# The DEM module

The dem module handles the pre-processing of bathymetric data. 


- Requires:

data folder. See [README](README.md) in this folder.

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Not neccessarily needed. Just to check version. Must be > 0.4.1
import pyPoseidon
pyPoseidon.__version__

In [ ]:
import pyPoseidon.dem as pdem
import numpy as np

### Graphics

In [ ]:
import matplotlib.pyplot as plt
#optional
#plt.style.use('dark_background')

In [ ]:
%matplotlib widget

## Setup

The minimum information required is the geomerty boundary. In the most simple case that is a lat/lon box that defines the area of interest. Without loss of generality we select below Iceland as a test case. Feel free to modify the coordinates. 

In [ ]:
#define in a dictionary the properties of the model..
geometry={'lon_min':-25., # lat/lon window
     'lon_max':-9.,
     'lat_min':56.,
     'lat_max':74.,
    }

In [ ]:
dem = pdem.dem(**geometry)

In [ ]:
dem.Dataset

In [ ]:
dem.Dataset.to_netcdf('./test/dem.nc') # save the file locally to be used below

## Specify Coastlines

The resolution of dem is usually less that the available coastline datasets.

Since the coastlines are the boundary for the grid generation it makes sense to have them matched to the bathymetric data. 

pyPoseidon tries to provide that (by default) when you provide a coastline dataset as argument.

In [ ]:
# use cartopy to get coastlines
import cartopy.feature as cf

cr='i'

coast = cf.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='{}m'.format({'l':110, 'i':50, 'h':10}[cr]))

In [ ]:
import geopandas as gp
ne_i = gp.GeoDataFrame(geometry = [x for x in coast.geometries()])

In [ ]:
ne_i.boundary.plot()

- Retrieve again the bathymetry providing also the coastlines as argument. This time we use a local dem file.

In [ ]:
dem = pdem.dem(**geometry, coastlines = ne_i, dem_source='./test/dem.nc')

In [ ]:
dem.Dataset

We can see the discrepancy in our lat/lon window between dem and coasts

In [ ]:
plt.figure()
dem.Dataset.elevation.where(dem.Dataset.elevation>0).plot()
aa = plt.gca()
ne_i.boundary.plot(ax=aa, color='orange')

However, things are better with the adjusted values

In [ ]:
plt.figure()
dem.Dataset.adjusted.where(dem.Dataset.adjusted>0).plot()
aa = plt.gca()
ne_i.boundary.plot(ax=aa, color='orange')

## Interpolate on Grid

In order to create the model, bathymetry data need to be interpolated on grid.
Let's load a SCHISM grid.

In [ ]:
import pyPoseidon.grid as pg

In [ ]:
g = pg.grid(type='tri2d',grid_file='./data/hgrid.gr3')

In [ ]:
g.Dataset

In [ ]:
# Get the nodes
x = g.Dataset.SCHISM_hgrid_node_x[:].values
y = g.Dataset.SCHISM_hgrid_node_y[:].values

We can now use the grid's x,y to get the interpolated values.

In [ ]:
dem = pdem.dem(**geometry, coastlines = ne_i, dem_source='./test/dem.nc', grid_x = x, grid_y = y)

In [ ]:
dem.Dataset

## Output

*Note* that SCHISM doesn't have a DEM file (it is included in the grid). The output here works only for D3D. 

We can create a simple 2D grid like

In [ ]:
## lat,lon grid
resolution=.1
lon=np.arange(geometry['lon_min'],geometry['lon_max'],resolution)
lat=np.arange(geometry['lat_min'],geometry['lat_max'],resolution)
xp, yp = np.meshgrid(lon,lat)

In [ ]:
d = pdem.dem(**geometry, coastlines = ne_i, dem_source='./test/dem.nc', grid_x = xp, grid_y = yp)

In [ ]:
d.Dataset

In [ ]:
pdem.to_output(d.Dataset,solver='d3d', rpath='./test/')